# Building with AI Agents

## What is an AI Agent?
An **AI agent** is a system that perceives its environment, processes information, and takes actions to achieve specific goals. AI agents use algorithms, models, and decision-making frameworks to interact with their surroundings intelligently.

This is different from applications such as ChatGPT which are not able to fulfil the "do action" criteria of AI Agents, for example, load a website and schedule posts. Note, LLMs are the "brains" for AI Agents, better LLMs will provide agents with better reasoning, long-term memory, multimodality, etc.

## Key Components of an AI Agent
1. **Perception**: Gathers data from the environment using sensors, APIs, or databases.
2. **Processing & Reasoning**: Uses machine learning models, logic-based algorithms, or heuristics to analyze data and make decisions.
3. **Action**: Executes tasks based on the analysis, interacting with the environment.
4. **Learning**: Adapts based on past experiences, using reinforcement learning, supervised learning, or unsupervised learning.

## Types of AI Agents
1. **Reactive Agents**: Respond to stimuli without storing past experiences (e.g., chess-playing AI).
2. **Model-Based Agents**: Maintain internal representations of the environment to predict future states.
3. **Goal-Based Agents**: Take actions that lead to predefined objectives.
4. **Utility-Based Agents**: Optimize decisions based on a utility function for best outcomes.
5. **Learning Agents**: Continuously improve by learning from interactions with the environment.

## Applications of AI Agents
- **Autonomous Vehicles**: AI agents process sensor data to make driving decisions.
- **Chatbots & Virtual Assistants**: Agents like ChatGPT interact with users and provide responses.
- **Game AI**: Non-player characters (NPCs) use AI agents to behave intelligently in games.
- **Financial Trading**: AI agents analyze market data to make automated trades.
- **Healthcare**: AI-powered diagnostics and personalized medicine.

## AI Agent Frameworks
- Most popular:
    - AutoGPT (first widely adopted agent framework)
    - BabyAGI
    - CrewAI
    - AutoGen
    - MultiOn
    - ChatDev
    - MetaGPT
- CrewAI
    - Perfect for begineers
    - No complex setup
    - Detailed documentation, fully open-sourced with the developers constantly providing updates

# AI Agents with CrewAI

First, we install the required libraries.

In [1]:
!pip install -q crewai crewai-tools langchain langchain_community python-dotenv

Import the required libraries

In [2]:
import os

from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

Import the API Keys

In [3]:
# Load environment variables from the .env file (if present)
load_dotenv()

# Access environment variables as if they came from the actual environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPER_API_KEY = os.getenv('SERPER_API_KEY')

Load Serper search tool and create agent.

 Serper is a low-cost Google Search API that can be used to add answer box, knowledge graph, and organic results data from Google Search.

In [4]:
search_tool = SerperDevTool()

`allow_delegation` this is one of the key differenciators of the CrewAI framework. It allows you to disable deligations for the agent as a Researcher should be doing the research instead of deligating to another agent. 

`tool` gives the agent access to all the different tools such as the search tool. This is the key differenciator with using something such as ChatGPT.

In [6]:
researcher = Agent (
    role = 'Senior Research Assistant',
    goal = 'Look up the latest Advancements in AI Agents',
    backstory = """You work at a leading tech think tank. Your expertise lies in searching Google for AI Agent frameworks.
                    You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.3)
)